#### Import Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


#### Foursquare API Credentials

In [2]:
CLIENT_ID = '3JT4PRZ23P0MIAB1BRBJJYEGF2SASKIZBHVDHQ2KNZZZQEEU'
CLIENT_SECRET = '1ZCCPVSUPZDSZASHEHPXTRVTZHDB2A5C3PQAXYVCJHYXBYMN'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3JT4PRZ23P0MIAB1BRBJJYEGF2SASKIZBHVDHQ2KNZZZQEEU
CLIENT_SECRET:1ZCCPVSUPZDSZASHEHPXTRVTZHDB2A5C3PQAXYVCJHYXBYMN


#### Load Neighbordhood Locations
This is the CSV file created in QGIS. Let's load the first 5 results, using the head function:

In [3]:
city_neighborhoods = pd.read_csv("dubai_neighborhoods_fixed6.csv")
city_neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Motion Gate,24.918613,55.006647
1,Jebel Ali Freezone,24.985820,55.045641
2,Jebel Ali Freezone Extension,24.956104,55.104863
3,Jebel Ali Industrial Area,24.996322,55.121104
4,Bluewaters Island,25.079605,55.122113


Let's verify our neighborhood locations on the map:

In [4]:
#Let's define our Dubai Centerpoint, which can be used later:
city_lat = 25.029435
city_lon = 55.272947
#Let's define our neighborhood map and print it:
city_map = folium.Map(location = [city_lat, city_lon], zoom_start = 10, tiles='openstreetmap')
for lat, lng, label in zip(city_neighborhoods['Latitude'], city_neighborhoods['Longitude'], city_neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(city_map)
city_map

Let's check the location of the first neighborhood:

In [5]:
neighborhood_latitude = city_neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = city_neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = city_neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Motion Gate are 24.91861281, 55.00664679.


#### Extract data from Foursquare
Now, let's get extract the top 5 venues from the Fourquare API:

In [6]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=3JT4PRZ23P0MIAB1BRBJJYEGF2SASKIZBHVDHQ2KNZZZQEEU&client_secret=1ZCCPVSUPZDSZASHEHPXTRVTZHDB2A5C3PQAXYVCJHYXBYMN&v=20180604&ll=24.91861281,55.00664679&radius=500&limit=100'

In [7]:
results = requests.get(url).json()

In [8]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,RIVERLAND,Plaza,24.919376,55.005381
1,Motiongate,Theme Park,24.920137,55.005107
2,French Village,Theme Park,24.917342,55.007796
3,Kalea,Hotel,24.916323,55.005876
4,McDonald's,Fast Food Restaurant,24.916865,55.007763


In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
city_venues = getNearbyVenues(names=city_neighborhoods['Neighborhood'],
                                   latitudes=city_neighborhoods['Latitude'],
                                   longitudes=city_neighborhoods['Longitude']
                                  )

Motion Gate
Jebel Ali Freezone 
Jebel Ali Freezone Extension 
Jebel Ali Industrial Area
Bluewaters Island
Ibn Battuta Mall/The Gardens
Jumeirah Beach Residences West
Dubai Marina West
Discovery Gardens North
Jumeirah Beach Residences East
Dubai Marina Mall
Jumeirah Lake Towers West
Palm Jumeirah
Dubai Marina East
Jumeirah Lake Towers East
Al Furjan
Meadows 2
Discovery Gardens South
Dubai Investment Park 1
Jumeirah Islands
Dubai Internet City
Jumeirah Park
Green Community
Emirates Hills 2
Knowledge Village
The Lakes
The Greens
Emirates Hills
Springs
Barsha Heights
Al Sufouh
Dubai Production City
Jumeirah Village Triangle
Umm Suqeim 3
Al Barsha 1
Al Barsha 2
Dubai Investment Park 2
Jumeirah Golf Estate
Umm Al Sheif
Umm Suqeim 2
Jumeirah Village Circle
Al Manara
Dubai Sports City
Al Quoz Industrial Area 3
Umm Suqeim 1
Jumeirah 3
Al Barsha South
Al Quoz Industrial Area 1
Al Quoz Industrial Area 4
Motor City
Al Quoz 3
Al Safa
Jumeirah
Al Quoz Industrial Area 2
Al Quoz 4
Al Quoz 1
Jumeirah 1

In [12]:
print(city_neighborhoods.shape)
city_neighborhoods.head()

(141, 3)


,Neighborhood,Latitude,Longitude
0,Motion Gate,24.918613,55.006647
1,Jebel Ali Freezone,24.985820,55.045641
2,Jebel Ali Freezone Extension,24.956104,55.104863
3,Jebel Ali Industrial Area,24.996322,55.121104
4,Bluewaters Island,25.079605,55.122113


In [13]:
city_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abu Hail,4,4,4,4,4,4
Academic City,1,1,1,1,1,1
Al Bada'a,7,7,7,7,7,7
Al Barsha 1,34,34,34,34,34,34
Al Barsha 2,3,3,3,3,3,3
Al Fahidi,21,21,21,21,21,21
Al Furjan,5,5,5,5,5,5
Al Hudaiba,11,11,11,11,11,11
Al Jaffiliya,8,8,8,8,8,8


In [14]:
print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))

There are 266 uniques categories.


<a id='item3'></a>

In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

47 venues were returned by Foursquare.


#### One Hot Encoding Process

In [16]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['Neighborhood'] = city_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,Zoo Exhibit,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cruise,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Paintball Field,Pakistani Restaurant,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Tram Station,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [17]:
city_onehot.shape

(2238, 266)

In [18]:
city_grouped = city_onehot.groupby('Neighborhood').mean().reset_index()
city_grouped

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cruise,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Paintball Field,Pakistani Restaurant,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Tram Station,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Abu Hail,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0

In [19]:
city_grouped.shape

(133, 266)

In [20]:
num_top_venues = 5

for hood in city_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = city_grouped[city_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abu Hail----
         venue  freq
0   Restaurant  0.50
1          Spa  0.25
2        Track  0.25
3  Opera House  0.00
4  Pastry Shop  0.00


----Academic City----
                     venue  freq
0             Burger Joint   1.0
1              Zoo Exhibit   0.0
2       Persian Restaurant   0.0
3  New American Restaurant   0.0
4                Nightclub   0.0


----Al Bada'a----
                       venue  freq
0                       Park  0.29
1          Indian Restaurant  0.14
2  Middle Eastern Restaurant  0.14
3                       Pool  0.14
4                Tailor Shop  0.14


----Al Barsha 1----
                       venue  freq
0                      Hotel  0.21
1  Middle Eastern Restaurant  0.12
2            Bed & Breakfast  0.09
3                 Restaurant  0.09
4                Art Gallery  0.03


----Al Barsha 2----
              venue  freq
0         Cafeteria  0.33
1         Bookstore  0.33
2        Food Court  0.33
3       Zoo Exhibit  0.00
4  Pedestrian Plaza  

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = city_grouped['Neighborhood']

for ind in np.arange(city_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,Restaurant,Track,Spa,Fish Market,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market
1,Academic City,Burger Joint,Zoo,Farm,French Restaurant,Fountain,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop
2,Al Bada'a,Park,Tailor Shop,Pool,Indian Restaurant,Café,Middle Eastern Restaurant,Fish & Chips Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant
3,Al Barsha 1,Hotel,Middle Eastern Restaurant,Bed & Breakfast,Restaurant,Bakery,Coffee Shop,Sandwich Place,Salad Place,Mediterranean Restaurant,Chinese Restaurant
4,Al Barsha 2,Bookstore,Food Court,Cafeteria,Zoo,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop


<a id='item4'></a>

In [23]:
city_neighborhoods
# set number of clusters
kclusters = 8

city_grouped_clustering = city_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 4, 0, 1, 1, 4, 1, 1], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = city_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
city_merged = city_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='right')

city_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Motion Gate,24.918613,55.006647,1,Theme Park Ride / Attraction,Theme Park,Italian Restaurant,Hotel,Café,Restaurant,Fast Food Restaurant,Coffee Shop,Plaza,Theater
1,Jebel Ali Freezone,24.985820,55.045641,1,Seafood Restaurant,Zoo,Fountain,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market
2,Jebel Ali Freezone Extension,24.956104,55.104863,5,Cruise,Falafel Restaurant,French Restaurant,Fountain,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market
3,Jebel Ali Industrial Area,24.996322,55.121104,3,Warehouse Store,Flower Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Zoo,Falafel Restaurant
4,Bluewaters Island,25.079605,55.122113,1,Restaurant,Resort,Bistro,Bar,Beach,Mediterranean Restaurant,Café,Scenic Lookout,Flea Market,Flower Shop


In [25]:
#Let's create our cluster map
map_clusters = folium.Map(location=[city_lat, city_lon], zoom_start=11, tiles="Stamen Terrain")

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['Latitude'], city_merged['Longitude'], city_merged['Neighborhood'], city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[cluster-3],
        fill=True,
        fill_color=rainbow[cluster-3],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
city_merged.loc[city_merged['Cluster Labels'] == 0, city_merged.columns[[1] + list(range(5, city_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,25.137126,Breakfast Spot,Basketball Court,Diner,Cafeteria,Flower Shop,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop
35,25.093816,Food Court,Cafeteria,Zoo,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop
55,25.170193,Bakery,Indian Restaurant,Harbor / Marina,Grocery Store,Ice Cream Shop,BBQ Joint,Flower Shop,Food & Drink Shop,Food Court
111,25.198289,Grocery Store,Basketball Court,Café,Zoo,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Fried Chicken Joint
114,25.224586,Grocery Store,Café,Cafeteria,Zoo,Fish Market,Filipino Restaurant,Fish & Chips Shop,Flea Market,Fried Chicken Joint
121,25.271522,Cafeteria,Market,Café,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Zoo,Farmers Market
131,25.186325,Zoo,Falafel Restaurant,Fountain,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market
134,25.266842,Zoo,Farm,French Restaurant,Fountain,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop


In [28]:
city_merged.loc[city_merged['Cluster Labels'] == 1, city_merged.columns[[1] + list(range(5, city_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,24.918613,Theme Park,Italian Restaurant,Hotel,Café,Restaurant,Fast Food Restaurant,Coffee Shop,Plaza,Theater
1,24.985820,Zoo,Fountain,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market
4,25.079605,Resort,Bistro,Bar,Beach,Mediterranean Restaurant,Café,Scenic Lookout,Flea Market,Flower Shop
5,25.043276,Coffee Shop,Egyptian Restaurant,Kids Store,Shoe Store,Middle Eastern Restaurant,Shopping Mall,Café,Salon / Barbershop,Bookstore
6,25.075448,Coffee Shop,Hotel,Lounge,Italian Restaurant,Middle Eastern Restaurant,Beach,Burger Joint,Hookah Bar,Restaurant
8,25.044083,Coffee Shop,Bus Stop,North Indian Restaurant,Basketball Court,Sandwich Place,Gym / Fitness Center,Trail,Lounge,Pizza Place
9,25.082086,Hotel,Italian Restaurant,Asian Restaurant,Burger Joint,Restaurant,Coffee Shop,Beach,Breakfast Spot,Yoga Studio
10,25.076211,Hotel,Restaurant,Café,Middle Eastern Restaurant,Massage Studio,Burger Joint,Lounge,Persian Restaurant,Pool
11,25.068645,Café,Restaurant,Italian Restaurant,Salad Place,Thai Restaurant,Bakery,Indian Restaurant,Pizza Place,Gym
12,25.109801,Café,Restaurant,Beach,Japanese Restaurant,Gym,Spa,Pool,Hotel,Convenience Store


In [29]:
city_merged.loc[city_merged['Cluster Labels'] == 2, city_merged.columns[[1] + list(range(5, city_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
116,25.102504,Zoo,Farm,French Restaurant,Fountain,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop
126,25.204606,Bakery,Zoo,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop


In [30]:
city_merged.loc[city_merged['Cluster Labels'] == 3, city_merged.columns[[1] + list(range(5, city_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,24.996322,Flower Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Zoo,Falafel Restaurant


In [31]:
city_merged.loc[city_merged['Cluster Labels'] == 4, city_merged.columns[[1] + list(range(5, city_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,25.071905,Café,Coffee Shop,Massage Studio,Moroccan Restaurant,Gym / Fitness Center,Tram Station,Harbor / Marina,Sushi Restaurant,Salon / Barbershop
26,25.092908,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Café,Gym,Gym Pool,Residential Building (Apartment / Condo),Grocery Store,Lake
29,25.098154,Nightclub,Middle Eastern Restaurant,Bar,Coffee Shop,Pub,Café,Pizza Place,Indian Restaurant,Thai Restaurant
34,25.111482,Middle Eastern Restaurant,Bed & Breakfast,Restaurant,Bakery,Coffee Shop,Sandwich Place,Salad Place,Mediterranean Restaurant,Chinese Restaurant
36,24.975227,Zoo,Farm,French Restaurant,Fountain,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop
39,25.150226,Convenience Store,Hotel,Zoo,Fish Market,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market
62,25.182640,Middle Eastern Restaurant,Coffee Shop,Italian Restaurant,Spa,Café,Bakery,French Restaurant,Lounge,Cocktail Bar
67,25.243365,Gym,Dessert Shop,Nightclub,Indian Restaurant,Convenience Store,Middle Eastern Restaurant,Lounge,Smoke Shop,Café
69,25.222290,French Restaurant,Nightclub,Restaurant,Lounge,Greek Restaurant,Residential Building (Apartment / Condo),Spa,Pizza Place,Café
70,25.255949,Indian Restaurant,Chinese Restaurant,Nightclub,North Indian Restaurant,Gastropub,Asian Restaurant,Bakery,Hookah Bar,Hotel Bar


In [32]:
city_merged.loc[city_merged['Cluster Labels'] == 5, city_merged.columns[[1] + list(range(5, city_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,24.956104,Falafel Restaurant,French Restaurant,Fountain,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market


In [33]:
city_merged.loc[city_merged['Cluster Labels'] == 6, city_merged.columns[[1] + list(range(5, city_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
119,25.262376,Boat or Ferry,Zoo,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop
128,25.188566,Zoo,Farm,French Restaurant,Fountain,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop


In [34]:
city_merged.loc[city_merged['Cluster Labels'] == 7, city_merged.columns[[1] + list(range(5, city_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
132,25.247048,Zoo,Falafel Restaurant,French Restaurant,Fountain,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop


#### Get Top 100 Breakfast Restaurants in Dubai.
After studying the Foursquare API, we know that there is a unique Category_ID field for Breakfast Spots: 4bf58dd8d48988d143941735. Let's extend the boundary of our search to a 30 KM radius. This is sufficient to cover the entire urban area of Dubai. Since we are using a free account on Foursquare, we must limit our venue search to 100, which is well enough for our study.

In [35]:
#Limit our search to 100 restaurants:
LIMIT_bf = 100
#Extend the boundary of our search to a 30 km radius:
radius_bf = 30000
#Get the url for breakfast spots and print:
category_ID = '4bf58dd8d48988d143941735'
url_bf = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_lat, 
    city_lon, 
    category_ID,
    radius_bf, 
    LIMIT_bf)
url_bf

'https://api.foursquare.com/v2/venues/explore?&client_id=3JT4PRZ23P0MIAB1BRBJJYEGF2SASKIZBHVDHQ2KNZZZQEEU&client_secret=1ZCCPVSUPZDSZASHEHPXTRVTZHDB2A5C3PQAXYVCJHYXBYMN&v=20180604&ll=25.029435,55.272947&categoryId=4bf58dd8d48988d143941735&radius=30000&limit=100'

In [36]:
results_bf = requests.get(url_bf).json()

In [37]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
venues_bf = results_bf['response']['groups'][0]['items']
    
nearby_venues_bf = json_normalize(venues_bf) # flatten JSON

# filter columns
filtered_columns_bf = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_bf =nearby_venues_bf.loc[:, filtered_columns_bf]

# filter the category for each row
nearby_venues_bf['venue.categories'] = nearby_venues_bf.apply(get_category_type, axis=1)

# clean columns
nearby_venues_bf.columns = [col.split(".")[-1] for col in nearby_venues_bf.columns]

nearby_venues_bf

,name,categories,lat,lng
0,Paul,French Restaurant,25.030099,55.270677
1,Town Square,Diner,25.009114,55.292299
2,Zaatar w Zeit Motor City,Middle Eastern Restaurant,25.046241,55.233530
3,Nathalie's,Bistro,25.038308,55.226764
4,Friends Avenue Cafe Motor City,Café,25.045927,55.239272
5,Shakespeare and Co. شكسبير أند كو,Breakfast Spot,25.056622,55.269935
6,The Cycle Bistro,Bistro,25.046555,55.238647
7,The Farm,Restaurant,25.094564,55.310409
8,Pizzeria Pulcinella,Breakfast Spot,25.037603,55.222217
9,Cafe Rider Custom Motorcycles,Café,25.117632,55.212545


In [39]:
#Let's define our breakfast spot map and print it:
bf_map = folium.Map(location = [city_lat, city_lon], zoom_start = 11, tiles='openstreetmap')
for lat, lng, label in zip(nearby_venues_bf['lat'], nearby_venues_bf['lng'], nearby_venues_bf['name'],):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='black',
        fill=True,
        fill_color='grey',
        fill_opacity=0.8,
        parse_html=False).add_to(map_clusters)
map_clusters

<a id='item5'></a>